<a href="https://colab.research.google.com/github/belladu0201/AIPI531_Deep-Reinforcement-Learning-and-Applications_SP23/blob/main/AIPI531_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 531: Deep Reinforcement Learning and Applications (SPRING 23)
## HW3
Name: Beibei Du

In [ ]:
# pip install d3rlpy

### 1. Build a pipeline for offline RL using d3rlpy:

Hint: In this homework, you will implement a simple pipeline for
offline RL using the APIs provided in d3rlpy repository.

[Required] Train CQL:
- Pick (any) one environment/dataset in d3rlpy
(https://github.com/takuseno/d3rlpy).
- Train your CQL agent with the data from the behavior policy.
- Generate true Q vs training steps (may include different
dataset sizes, task difficulties) plots.
- Generate estimated Q vs training steps (may include
different dataset sizes, task difficulties) plots.

In [ ]:
import d3rlpy

from d3rlpy.datasets import get_cartpole

# get CartPole dataset
dataset, env = get_cartpole()

from d3rlpy.algos import DiscreteCQL
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

# split train and test episodes
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

# start training
cql.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=1,
        scorers={
            'environment': evaluate_on_environment(env), # evaluate with CartPol-v0 environment
            'advantage': discounted_sum_of_advantage_scorer, # smaller is better
            'td_error': td_error_scorer, # smaller is better
            'value_scale': average_value_estimation_scorer # smaller is better
        })

/usr/local/lib/python3.9/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


2023-03-22 23:36:06 [debug    ] RoundIterator is selected.
2023-03-22 23:36:06 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20230322233606
2023-03-22 23:36:06 [debug    ] Building models...
2023-03-22 23:36:06 [debug    ] Models have been built.
2023-03-22 23:36:06 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20230322233606/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 8000, 'use_gpu': None, 'algorithm': 'DiscreteCQL', 'observation

Epoch 1/1:   0%|          | 0/2454 [00:00<?, ?it/s]

2023-03-22 23:36:29 [info     ] DiscreteCQL_20230322233606: epoch=1 step=2454 epoch=1 metrics={'time_sample_batch': 0.0002179554835897009, 'time_algorithm_update': 0.00696141708159505, 'loss': 0.6871370229423774, 'time_step': 0.007341974036132017, 'environment': 200.0, 'advantage': -3.0136605143338078, 'td_error': 1.1679924397038388, 'value_scale': 1.1233491502113153} step=2454
2023-03-22 23:36:29 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20230322233606/model_2454.pt


[(1,
  {'time_sample_batch': 0.0002179554835897009,
   'time_algorithm_update': 0.00696141708159505,
   'loss': 0.6871370229423774,
   'time_step': 0.007341974036132017,
   'environment': 200.0,
   'advantage': -3.0136605143338078,
   'td_error': 1.1679924397038388,
   'value_scale': 1.1233491502113153})]

In [ ]:
import glob
import io
import base64

from gym.wrappers import RecordVideo
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

# start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

# play recorded video
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

FileNotFoundError: ignored

In [ ]:
# wrap RecordVideo wrapper
env = RecordVideo(env, './video')

# evaluate
evaluate_on_environment(env)(cql)

show_video()

In [ ]:
pip install pyvirtualdisplay

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Reference
[1]. https://github.com/takuseno/d3rlpy/blob/master/tutorials/cartpole.ipynb

[2]. https://github.com/takuseno/d3rlpy

[3].